In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome import service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException as WDE
from selenium.common.exceptions import NoSuchElementException, InvalidSessionIdException


import pandas as pd
import time

In [7]:
isbns = pd.read_csv('isbns.csv')
isbns
theme = isbns.columns.tolist()
isbns_ll = isbns.values.tolist()
len(isbns_ll[0])

1404

In [9]:
# theme

In [4]:
# parser.py
import requests
from bs4 import BeautifulSoup as bs

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [5]:
options = Options()
options.add_argument("--no-sandbox") #bypass OS security model
options.add_argument("--headless")
options.add_argument("--disable-dev-shm-usage")

In [6]:
from selenium.webdriver.common.by import By


> # 다 파싱

In [66]:
isbns_ll[1][0]

9788932909707

In [65]:
len(isbns_ll)

6

In [12]:
samples = isbns_ll[0][:10]
samples

[9788983928061,
 9788983928030,
 9788983928054,
 9791131992272,
 9788937460258,
 9788983927705,
 9791133440054,
 9788992711937,
 9788983927682,
 9791191667103]

In [22]:
# theme

In [23]:
import re

breakpoints = None
nara = ['테마','고전','시/희곡', '비평/창작/이론', '한국','영미']

for genreidx,genre in enumerate(isbns_ll):
    # ll = []
    data = pd.DataFrame()
    s=Service('/opt/ml/chromedriver')
    driver = webdriver.Chrome(options=options, service=s)
    driver.implicitly_wait(5)
    keywordpick = []
    keywords = []
    isbn_t = []
    description = []
    mokcha = []
    catchphrase = []
    title = []
    author = []
    image = []
    for idx,isbn in enumerate(tqdm(genre)):
        genre_info = nara[genreidx]
        isbn_t.append(isbn)
        url = "http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=" + str(isbn) + "&orderClick=JAI"
        if idx % 100 == 99:
            driver.quit()
            s=Service('/opt/ml/chromedriver')
            driver = webdriver.Chrome(options=options, service=s)
            driver.implicitly_wait(5)
        driver.get(url)
       
        # TODO keywordpick
        keywordpick_ll = []
        try:
            keywordsss = driver.find_element_by_class_name("book_keyword")
            for i in keywordsss.find_elements_by_xpath('.//*'):
                keywordpick_ll.append(i.text)
        except NoSuchElementException as n:
            keywordpick_ll=[]
        keywordpick.append(keywordpick_ll)
        # TODO author
        author_ll = []
        # author = []
        try:
            name = driver.find_element_by_class_name("name")
            author_ll.append(name.text)
        except NoSuchElementException as n:
            author_ll = []
        # onebookdict['author'] = author_ll
        # if(len(author) == 0):
        #     author_ll=[]
        author.append(author_ll)
        # TODO title
        title_ll=[]
        try:
            titles = driver.find_element_by_xpath('//*[@id="container"]/div[2]/form/div[1]/h1')
            title_ll.append(titles.text)
        except NoSuchElementException as n:
            title_ll=[]
        # onebookdict['title'] = title_ll
        # print(title_ll)
        title.append(title_ll)
        # TODO image
        image_ll=[]
        try:
            images = driver.find_element_by_xpath('//*[@id="container"]/div[2]/form/div[2]/div[1]/div/a/img').get_attribute("src") # 크게 뜬 이미지 선택하여 "src" 속성을 받아옴
            image_ll.append(images)
        except NoSuchElementException as n:
            image_ll=[]
        image.append(image_ll)
        # TODO tag_list(주제어)
        # tag_ll=[]
        try:
            tag = driver.find_elements_by_class_name("tag_list")
            if(len(tag) != 0):
                temp = re.sub('#', '', tag[0].text)
                tags = temp.split(' ')
            else:
                tags = []
        
            # tag_ll.append()
        except NoSuchElementException as n:
            tags = []
        keywords.append(tags)

        # TODO 책소개, 책 속으로, 목차 한꺼번에 버튼클릭은 해결못함
        # title_detail_basic2
        # box_detail_article

        # box_detail_comment
        description_ll=[]
        mokcha_ll=[]
        try:
            detail = driver.find_elements_by_class_name("box_detail_article")     
            for i in detail:
                if(len(i.text)!=0):
                    if(i.text != '이 도서는 목차가 없습니다.' and i.text != '이 상품은 목차가 없습니다.'):
                        if(i.text[0].isdigit() or i.text[0] == '제'):
                            # TODO 목차정보
                            mokcha_ll.append(i.text)
                        else:
                            description_ll.append(i.text)
        except NoSuchElementException as n:
            mokcha_ll = []
            description_ll = []
        mokcha.append(mokcha_ll)
        description.append(description_ll)

        # TODO Catchphrase
        catch_ll=[]
        try:
            detail2 = driver.find_elements_by_class_name("title_detail_basic2")
            if len(detail2) != 0:
                for i in detail2:
                    if(i.text != '북소믈리에 한마디!'):
                        catch_ll.append(i.text)
                        # print(i.text)
                # onebookdict['Catchphrase'] = catch_ll
            else:
                catch_ll=[]
                # onebookdict['Catchphrase'] = [None]
        except NoSuchElementException as n:
            catch_ll=[]
        catchphrase.append(catch_ll)

    data['author'] = author
    data['title'] = title
    data['image'] = image
    data['keywordpick'] = keywordpick
    data['keywords'] = keywords
    # data['text'] = text
    data['isbn'] = isbn_t
    data['mokcha'] = mokcha
    data['description'] = description
    data['catchphrase'] = catchphrase

    driver.quit()
    data.to_csv("{}_kyobo.csv".format(genre_info), index=False)

#     # break



# TODO 장르소설
data = pd.DataFrame()
s=Service('/opt/ml/chromedriver')
driver = webdriver.Chrome(options=options, service=s)
driver.implicitly_wait(5)
keywordpick = []
keywords = []
isbn_t = []
description = []
mokcha = []
catchphrase = []
title = []
author = []
image = []
for idx, isbn in enumerate(tqdm(theme[:10])):
    genre_info = '장르소설'
    isbn_t.append(isbn)
    
    url = "http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=" + str(isbn) + "&orderClick=JAI"
    if idx % 100 == 99:
        driver.quit()
        s=Service('/opt/ml/chromedriver')
        driver = webdriver.Chrome(options=options, service=s)
        driver.implicitly_wait(5)
    driver.get(url)
    

    # TODO author
    author_ll = []
    # author = []
    try:
        name = driver.find_element_by_class_name("name")
        author_ll.append(name.text)
    except NoSuchElementException as n:
        author_ll = []
    # onebookdict['author'] = author_ll
    # if(len(author) == 0):
    #     author_ll=[]
    author.append(author_ll)

    # TODO title
    title_ll=[]
    try:
        titles = driver.find_element_by_xpath('//*[@id="container"]/div[2]/form/div[1]/h1')
        title_ll.append(titles.text)
    except NoSuchElementException as n:
        title_ll=[]
    # onebookdict['title'] = title_ll
    # print(title_ll)
    title.append(title_ll)

    # TODO image
    image_ll=[]
    try:
        images = driver.find_element_by_xpath('//*[@id="container"]/div[2]/form/div[2]/div[1]/div/a/img').get_attribute("src") # 크게 뜬 이미지 선택하여 "src" 속성을 받아옴
        image_ll.append(images)
    except NoSuchElementException as n:
        image_ll=[]
    image.append(image_ll)

    # TODO keywordpick
    keywordpick_ll = []
    try:
        keywordsss = driver.find_element_by_class_name("book_keyword")
        for i in keywordsss.find_elements_by_xpath('.//*'):
            keywordpick_ll.append(i.text)
    except NoSuchElementException as n:
        keywordpick_ll=[]
    keywordpick.append(keywordpick_ll)
    
    # TODO tag_list(주제어)
    # tag_ll=[]
    try:
        tag = driver.find_elements_by_class_name("tag_list")
        if(len(tag) != 0):
            temp = re.sub('#', '', tag[0].text)
            tags = temp.split(' ')
        else:
            tags = []
    
        # tag_ll.append()
    except NoSuchElementException as n:
        tags = []
    keywords.append(tags)
# TODO 책소개, 책 속으로, 목차 한꺼번에 버튼클릭은 해결못함
        # title_detail_basic2
        # box_detail_article

        # box_detail_comment
    description_ll=[]
    mokcha_ll=[]
    try:
        detail = driver.find_elements_by_class_name("box_detail_article")     
        for i in detail:
            if(len(i.text)!=0):
                if(i.text != '이 도서는 목차가 없습니다.' and i.text != '이 상품은 목차가 없습니다.'):
                    if(i.text[0].isdigit() or i.text[0] == '제'):
                        # TODO 목차정보
                        mokcha_ll.append(i.text)
                    else:
                        description_ll.append(i.text)
    except NoSuchElementException as n:
        mokcha_ll = []
        description_ll = []
    mokcha.append(mokcha_ll)
    description.append(description_ll)

    # TODO Catchphrase
    catch_ll=[]
    try:
        detail2 = driver.find_elements_by_class_name("title_detail_basic2")
        if len(detail2) != 0:
            for i in detail2:
                if(i.text != '북소믈리에 한마디!'):
                    catch_ll.append(i.text)
                    # print(i.text)
            # onebookdict['Catchphrase'] = catch_ll
        else:
            catch_ll=[]
            # onebookdict['Catchphrase'] = [None]
    except NoSuchElementException as n:
        catch_ll=[]
    catchphrase.append(catch_ll)

data['author'] = author
data['title'] = title
data['image'] = image
data['keywordpick'] = keywordpick
data['keywords'] = keywords
data['isbn'] = isbn_t
data['mokcha'] = mokcha
data['description'] = description
data['catchphrase'] = catchphrase

driver.quit()
data.to_csv("{}_kyobo.csv".format(genre_info), index=False)





  0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_32733/1316514767.py:176: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  name = driver.find_element_by_class_name("name")
/tmp/ipykernel_32733/1316514767.py:188: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  titles = driver.find_element_by_xpath('//*[@id="container"]/div[2]/form/div[1]/h1')
/tmp/ipykernel_32733/1316514767.py:199: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  images = driver.find_element_by_xpath('//*[@id="container"]/div[2]/form/div[2]/div[1]/div/a/img').get_attribute("src") # 크게 뜬 이미지 선택하여 "src" 속성을 받아옴
/tmp/ipykernel_32733/1316514767.py:208: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  keywordsss = driver.find_element_b

In [24]:
data

,author,title,image,keywordpick,keywords,isbn,mokcha,description,catchphrase
0,[요코제키 다이],[루팡의 딸 2 루팡의 귀환],[http://image.kyobobook.co.kr/images/book/larg...,"[일본소설, 미스터리소설, 추리소설, 미스터리, 수사, 도둑]","[미스터리소설, 추리소설, 일본소설]",9791190157346,[],[도둑 집안의 딸 하나코와 경찰 집안의 아들 카즈마는 사랑스러운 딸 안을 낳고 평범...,[루팡의 딸이 결혼하고 돌아왔다!]
1,[J. K. 롤링],[해리 포터와 마법사의 돌(미나리마 에디션)],[http://image.kyobobook.co.kr/images/book/larg...,"[성장소설, 영미소설, 판타지소설, 영화원작소설, 마법 세계, 영국소설]","[영미소설, 영국소설, 판타지소설, 영화원작소설, 성장소설, 마법, 호그와트, 헤르...",9788983928207,[제1장 살아남은 아이\n제2장 사라진 유리창\n제3장 발신자 없는 편지들\n제4장...,[영화 〈해리 포터〉와 〈신비한 동물사전〉 시리즈에서 비주얼 그래픽을 담당했던 스튜...,[《해리 포터와 마법사의 돌: 미나리마 에디션》 전 세계 동시 출간!\n〈해리 포터...
2,[야쿠마루 가쿠],[돌이킬 수 없는 약속],[http://image.kyobobook.co.kr/images/book/larg...,"[소설, 미스터리, 미스터리소설, 교도소, 과거, 편지지]",[미스터리소설],9788998274795,[],"[주인공이 과거에 저지른 죄, 그리고 15년 전에 했던 어떤 약속을 둘러싸고 벌어지...",[응징과 용서의 진정한 의미는 무엇인가!]
3,[싱숑],[전지적 독자 시점 PART 1(제4의 벽 북스탠드 에디션 세트)],[http://image.kyobobook.co.kr/images/book/larg...,"[한국소설, 판타지소설, 공포, 웹 소설, 네이버 웹툰, 페이퍼백]","[한국소설, 판타지소설, 웹소설, 현실, 혼돈, 공포, 결말, 진실, 예언]",9788934967576,[],"[“이 이야기를 모두 읽고 난 뒤, 당신은 오래도록 잊고 지낸 기억 하나를 되찾는다...",[☆☆☆‘네이버 시리즈’ 누적 다운로드 1억☆☆☆\n☆☆☆‘문피아’ 누적 판매 1위...
4,[사노 테츠야],[너는 달밤에 빛나고],[http://image.kyobobook.co.kr/images/book/larg...,"[마미, 전격소설대상, 시한부, 발광, 러브 스토리, 소녀]",[전격소설대상],9791127844745,[],[“이제 곧 마지막 순간이 다가옵니다. 이것이 정말 마지막 부탁입니다…….”\n\n...,[]
5,[스미노 요루],[너의 췌장을 먹고 싶어],[http://image.kyobobook.co.kr/images/book/larg...,"[베스트셀러, 청춘 소설, 공병, 일본 문단, 문고, 서점]",[],9791157107223,[2016년 일본 서점 대상 2위에 오른 스미노 요루의 첫 소설이다. 시한부 선고를...,[▶ 『너의 췌장을 먹고 싶어』북트레일러\n\n\n▶ 『너의 췌장을 먹고 싶어』 1...,[잔혹한 비밀을 가진 소녀와 임시 친구 계약을 맺은 소년의 이야기!]
6,[싱숑],[전지적 독자 시점 PART 1: 2 싱숑 장편소설],[http://image.kyobobook.co.kr/images/book/larg...,"[한국소설, 판타지소설, 공포, 웹 소설, 네이버 웹툰, 페이퍼백]","[한국소설, 판타지소설, 웹소설, 현실, 혼돈, 공포, 결말, 진실, 예언]",9788934967323,[],"[‘문피아’ 누적 판매 1위, ‘네이버 시리즈’ 누적 다운로드 1억, 웹툰화 즉시 ...",[토털 2억 뷰 + α! 웹소설의 현재 진행형 레전드\n마침내 단행본으로 만나는 《...
7,[프랭크 허버트],[듄 1-6 세트],[http://image.kyobobook.co.kr/images/book/larg...,"[영미소설, 영화원작소설, SF소설, 우주, 행성, 미국소설]","[네뷸러상, 휴고상, 영미소설, 미국소설, SF소설, 영화원작소설, 우주시대, 인류...",9791158887605,[제1권 듄 DUNE\n제2권 듄의 메시아 DUNE MESSIAH\n제3권 듄의 아...,[역사상 가장 많이 팔린 SF이자 독자들이 뽑은 역대 최고의 SF인 프랭크 허버트의...,"[""『듄』에 견줄 수 있는 건 『반지의 제왕』 외에는 없다."" -아서 C. 클라크\..."
8,[히가시노 게이고],[가면산장 살인사건],[http://image.kyobobook.co.kr/images/book/larg...,"[소설, 스릴러소설, 서스펜스, 별장, 일본추리소설, 작가]","[일본추리소설, 스릴러소설]",9788990982575,[],[히가시노 게이고의 소설 『가면산장 살인사건』. 저자와 독자가 아슬아슬한 두뇌 싸움...,[강도와 인질 간의 숨 막히는 줄다리기!]
9,[싱숑],[전지적 독자 시점 PART 1: 1 싱숑 장편소설],[http://image.kyobobook.co.kr/images/book/larg...,"[한국소설, 판타지소설, 공포, 웹 소설, 네이버 웹툰, 페이퍼백]","[한국소설, 판타지소설, 웹소설, 현실, 혼돈, 공포, 결말, 진실, 예언]",9788934967316,[],"[‘문피아’ 누적 판매 1위, ‘네이버 시리즈’ 누적 다운로드 1억, 웹툰화 즉시 ...",[토털 2억 뷰 + α! 웹소설의 현재 진행형 레전드\n마침내 단행본으로 만나는 《...
